<a href="https://colab.research.google.com/github/Omarakwah/Grad_Project/blob/main/KLAO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -U langgraph langchain_openai langchain_community
%pip install -qU langchain langchain-openai langchain-community langchain-experimental
!pip install -q langchain langchain-community langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
     

In [2]:
import re
import dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_core.output_parsers import StrOutputParser

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
Google_API_KEY=os.environ["GOOGLE_API_KEY"]

In [5]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.3,
    max_tokens=None,
    timeout=None,
    google_api_key=Google_API_KEY

)

In [6]:
from langchain_core.prompts import ChatPromptTemplate

template = """You are AI Assitant who expert in german language


 Your goal is to :
 Add contextual explanations to help readers understand large numbers better.
  Use visual comparisons to make abstract quantities more relatable.
to this {raw_text}
  Example:
 1 Million Euro → So viel Geld, dass man 100 Autos kaufen könnte
 10.000 Besucher → So viele Menschen, wie in ein großes Fußballstadion passen
  30 Prozent der Fläche → Ein Drittel, also ein Stück von drei gleich großen
 Teilen
 250 Kilogramm → So schwer wie ein großer Kühlschrank
Your answer shoud not exceed 1 line
Reply with complete sentence according to {raw_text} sentence
Please Don't invent numbers

"""
prompt = ChatPromptTemplate.from_template(template)

In [8]:
import re
from math import ceil
from langchain.chains import LLMChain

# Create the LangChain chain
chain = LLMChain(llm=llm, prompt=prompt)

def simplify_numbers(raw_text):
    def round_large_numbers(match):
        number = match.group()
        # Handle numbers with commas (e.g., "324.620,22")
        number = number.replace('.', '').replace(',', '.')
        number = float(number)
        rounded_number = None

        if number >= 1000:
            rounded_number = f"etwa {int(round(number, -3))}"
        elif number >= 10:
            rounded_number = f"etwa {ceil(number)}"
        else:
            rounded_number = f"etwa {round(number, 1)}"

        return rounded_number

    def simplify_percentages(match):
        percentage = float(match.group(1))
        if percentage == 25:
            return "jeder Vierte"
        elif percentage == 50:
            return "die Hälfte"
        elif percentage == 75:
            return "drei von vier"
        elif percentage >= 90:
            return "fast alle"
        elif percentage >= 60:
            return "mehr als die Hälfte"
        elif percentage <= 15:
            return "wenige"
        else:
            return f"etwa {percentage} Prozent"

    def contextualize_numbers(raw_text):
        return chain.run({"raw_text": raw_text})

    # Round large numbers
    raw_text = re.sub(r"\b\d{1,3}(?:\.\d{3})*(?:,\d+)?\b", round_large_numbers, raw_text)

    # Simplify percentages
    raw_text = re.sub(r"(\d+(?:\.\d+)?) Prozent", simplify_percentages, raw_text)

    # Handle contextual explanations and visual comparisons using LLM
    raw_text = contextualize_numbers(raw_text)

    return raw_text

# Test cases
test_cases = [
    "324.620,22 Euro wurden gespendet.",
    "1.897 Menschen nahmen teil.",
    "25 Prozent der Bevölkerung sind betroffen.",
    "90 Prozent stimmten zu.",
    "14 Prozent lehnten ab.",
    "Bei 38,7 Grad Celsius ist es sehr heiß.",
    "denn die Rente steigt um 4,57 Prozent.",
    "Im Jahr 2024 gab es 1.234 Ereignisse.",
    "Am 1. Januar 2024 waren es 5.678 Teilnehmer.",
    "Im Jahr 2025 gab es 2018 Ereignisse."
]

for test in test_cases:
    print(simplify_numbers(test))


<ipython-input-8-40a1e2a07a31>:43: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return chain.run({"raw_text": raw_text})


Etwa 325.000 Euro wurden gespendet – das ist ungefähr so viel, wie der Kaufpreis von 325 Kleinwagen.

Etwa 2000 Menschen nahmen teil – das entspricht in etwa der Größe zweier mittelgroßer Schulen oder mehreren vollen Reisebussen.

Etwa jeder Vierte der Bevölkerung ist betroffen – das entspricht ungefähr 25 Prozent, also so viel wie ein Viertel der Bevölkerung eines Landes.

Etwa fast alle stimmten zu –  eine überwältigende Mehrheit, vergleichbar mit einer nahezu voll besetzten Arena.

Etwa wenige lehnten ab –  nur ein kleiner Bruchteil, vergleichbar mit wenigen Sandkörnern auf einem Strand.

Bei etwa 39 Grad Celsius ist es sehr heiß, so heiß wie an einem sehr heißen Sommertag, an dem man sich kaum im Freien aufhalten möchte.

Denn die Rente steigt um einen geringen Betrag,  vergleichbar mit dem Preis eines kleineren Kaffees.

Im Jahr 2024 gab es etwa 1000 Ereignisse,  das entspricht in etwa der Anzahl von Veranstaltungen in einer mittelgroßen Stadt über ein Jahr.

Am etwa 1. Januar 202